Download semua requirements untuk model dibawah (Karena sudah dijalankan di Local Conda saya, ga dijalankan lagi)

In [2]:
%pip install pandas scikit-learn nltk sastrawi transformers torch accelerate numpy tf-keras

  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.2.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-an


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Checking the Tensorflow version to help debugging in the later stage.

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)

2.19.0
3.9.0


Importing and processing the data, so the data is standardized. Also included the stopwords removal.

In [2]:
import pandas as pd
import sklearn
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')

# Memuat dataset
data = pd.read_csv('Ulasan_Gojek_3000_rev.csv')

# Ganti nama kolom jika diperlukan
data.columns = ['review']

# Ekstrak fitur (teks ulasan) dan label
X_raw = data['review']

# Ubah teks menjadi huruf kecil
X_lower = X_raw.str.lower()

# Hapus angka dari teks
def hapus_angka(teks):
    teksTanpaAngka = ''.join([char for char in teks if not char.isdigit()])
    return teksTanpaAngka

X_tanpa_angka = X_lower.apply(hapus_angka)

# Hapus tanda baca dari teks
def hapus_tanda(teks):
    punctuation_set = set(string.punctuation)
    teksTanpaTanda = ''.join([char for char in teks if char not in punctuation_set])
    return teksTanpaTanda

X_tanpa_tanda = X_tanpa_angka.apply(hapus_tanda)

# Unduh stopwords dalam Bahasa Indonesia
nltk.download('stopwords')
nltk.download('punkt')
stopwords_indonesia = set(stopwords.words('indonesian'))

# Hapus stopwords dari teks
def hapus_stopwords(teks):
    # Tokenisasi teks menjadi kata-kata
    tokens_kata = word_tokenize(teks)
    kata_penting = [kata for kata in tokens_kata if kata not in stopwords_indonesia]

    # Gabungkan kata-kata penting kembali menjadi teks
    teks_tanpa_stopwords = ' '.join(kata_penting)
    return teks_tanpa_stopwords

X_bersih = X_tanpa_tanda.apply(hapus_stopwords)

print(X_tanpa_angka.head())
print(X_tanpa_tanda.head())
print(X_bersih.head())

# Simpan data yang telah diproses ke dalam file CSV baru
data_bersih = pd.DataFrame({'review': X_bersih})
data_bersih.to_csv('Ulasan_Gojek_3000_bersih.csv', index=False)

[nltk_data] Downloading package punkt_tab to C:\Users\Muhammad
[nltk_data]     Fachri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Muhammad
[nltk_data]     Fachri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Muhammad
[nltk_data]     Fachri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0    tolong yang kalo beli gojek plus harusnya lebi...
1    akhir ini tidak ada tanda posisi driver saat p...
2    bagus, sedikit kecewa krna promo hanya bisa di...
3    aplikasi sudah sangat baik, tapi saya kecewa, ...
4    saya ga masalah sama aps gojeknya, tapi saya g...
Name: review, dtype: object
0    tolong yang kalo beli gojek plus harusnya lebi...
1    akhir ini tidak ada tanda posisi driver saat p...
2    bagus sedikit kecewa krna promo hanya bisa dig...
3    aplikasi sudah sangat baik tapi saya kecewa ha...
4    saya ga masalah sama aps gojeknya tapi saya ga...
Name: review, dtype: object
0    tolong kalo beli gojek plus premiumkan benefit...
1    tanda posisi driver pesan gofood nggak posisin...
2    bagus kecewa krna promo memilih metode pembaya...
3    aplikasi kecewa gojek data riwayat transaksi p...
4    ga aps gojeknya ga suka iklan pop up gofood yg...
Name: review, dtype: object


This process is to stem and then tokenize the text, this helps to process the data so further processing is easier.

In [3]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Create the stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Function to clean, stem, and tokenize the text
def preprocess(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    stemmed_text = stemmer.stem(text)  # Apply stemming
    tokens = stemmed_text.split()  # Tokenize the text
    return tokens

# Read CSV file
df = pd.read_csv('Ulasan_Gojek_3000_bersih.csv')

# Apply preprocessing to the text column (assuming the text is in a column named 'text')
df['processed_text'] = df['review'].apply(preprocess)

# Save the processed text to a new CSV file (if needed)
df.to_csv('processed_file.csv', index=False)

# Display the first few rows to check the output
print(df.head())


                                              review  \
0  tolong kalo beli gojek plus premiumkan benefit...   
1  tanda posisi driver pesan gofood nggak posisin...   
2  bagus kecewa krna promo memilih metode pembaya...   
3  aplikasi kecewa gojek data riwayat transaksi p...   
4  ga aps gojeknya ga suka iklan pop up gofood yg...   

                                      processed_text  
0  [tolong, kalo, beli, gojek, plus, premium, ben...  
1  [tanda, posisi, driver, pesan, gofood, nggak, ...  
2  [bagus, kecewa, krna, promo, pilih, metode, ba...  
3  [aplikasi, kecewa, gojek, data, riwayat, trans...  
4  [ga, aps, gojek, ga, suka, iklan, pop, up, gof...  


Dibawah ini pemberian sentiment dengan file TSV source dari google.

In [4]:
import csv
import ast  # For safely evaluating string representations of lists

# Load lexicon from file
def load_lexicon(filename):
    lexicon = {}
    with open(filename, mode='r', encoding='utf-8') as file:
        tsv_reader = csv.reader(file, delimiter='\t')
        next(tsv_reader)  # Skip the header row
        for row in tsv_reader:
            word, weight = row[0], int(row[1])
            lexicon[word] = weight
    return lexicon

# Load positive and negative lexicons
positive_lexicon = load_lexicon('positive.tsv')
negative_lexicon = load_lexicon('negative.tsv')

# Combine positive and negative lexicons
lexicon = {**positive_lexicon, **negative_lexicon}

# Debugging: Check the lexicon contents
print("Lexicon sample:", list(lexicon.items())[:10])

# Function to calculate sentiment score
def calculate_sentiment(tokens, lexicon):
    score = 0
    for token in tokens:
        if token in lexicon:
            score += lexicon[token]
        else:
            print(f"Token not found in lexicon: {token}")  # Debugging: Print tokens not found in lexicon
    return score

# Load the processed CSV file
df = pd.read_csv('processed_file.csv')  # Replace with your actual file path

# Debugging: Check a sample of the processed text
print("Sample processed text:", df['processed_text'].head())

# Ensure `processed_text` is a list of tokens
df['processed_text'] = df['processed_text'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Calculate sentiment score for each row
df['sentiment_score'] = df['processed_text'].apply(lambda tokens: calculate_sentiment(tokens, lexicon))

# Debugging: Check the resulting sentiment scores
print("Sample sentiment scores:", df[['processed_text', 'sentiment_score']].head())

# Save the DataFrame with sentiment scores to a new CSV file
df.to_csv('processed_with_sentiment.csv', index=False)


Lexicon sample: [('hai', 3), ('merekam', 2), ('ekstensif', 3), ('paripurna', 1), ('detail', 2), ('pernik', 3), ('belas', -3), ('welas', -3), ('kabung', -5), ('rahayu', -2)]
Sample processed text: 0    ['tolong', 'kalo', 'beli', 'gojek', 'plus', 'p...
1    ['tanda', 'posisi', 'driver', 'pesan', 'gofood...
2    ['bagus', 'kecewa', 'krna', 'promo', 'pilih', ...
3    ['aplikasi', 'kecewa', 'gojek', 'data', 'riway...
4    ['ga', 'aps', 'gojek', 'ga', 'suka', 'iklan', ...
Name: processed_text, dtype: object
Token not found in lexicon: kalo
Token not found in lexicon: gojek
Token not found in lexicon: benefitnya
Token not found in lexicon: rb
Token not found in lexicon: ongkir
Token not found in lexicon: pake
Token not found in lexicon: ribu
Token not found in lexicon: karna
Token not found in lexicon: ratarata
Token not found in lexicon: gak
Token not found in lexicon: ama
Token not found in lexicon: gak
Token not found in lexicon: worth
Token not found in lexicon: it
Token not found in lexi

Klasifikasi menjadi positive, negative dan neutral.

In [5]:
# Load the processed CSV file
df = pd.read_csv('processed_with_sentiment.csv')  # Replace with your actual file path

# Define classification function based on sentiment_score
def classify_sentiment(score):
    if score < 0:
        return "Negative"
    elif score > 0:
        return "Positive"
    else:
        return "Neutral"

# Apply classification to the original sentiment scores
df['sentiment_label'] = df['sentiment_score'].apply(classify_sentiment)

# Debugging: Check the resulting classifications
print("Sample sentiment classifications:", df[['sentiment_score', 'sentiment_label']].head())

# Save the DataFrame with classified sentiment labels to a new CSV file
df.to_csv('processed_with_classified_sentiment.csv', index=False)

print("File saved successfully as 'processed_with_classified_sentiment.csv'")


Sample sentiment classifications:    sentiment_score sentiment_label
0              -12        Negative
1               -7        Negative
2              -19        Negative
3              -36        Negative
4                5        Positive
File saved successfully as 'processed_with_classified_sentiment.csv'


Normalized sentiment dibawah, menggunakan minmax karena tidak cocok pakai z-score.

NOTES: kode ini ga dipake karena sesuai dengan tujuan klasifikasi.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Load the processed CSV file
df = pd.read_csv('processed_with_sentiment.csv')  # Replace with your actual file path

# Normalization: Min-Max Scaling
scaler = MinMaxScaler()
df['normalized_score_minmax'] = scaler.fit_transform(df[['sentiment_score']])

# Debugging: Check the resulting normalized scores
print("Sample normalized sentiment scores:", df[['sentiment_score', 'normalized_score_minmax']].head())

# Save the DataFrame with sentiment scores and normalized scores to a new CSV file
df.to_csv('processed_with_normalized_sentiment.csv', index=False)

print("File saved successfully as 'processed_with_normalized_sentiment.csv'")

Debugging dan reinstall torch. hanya di run JIKA torch bermasalah, jika tidak skip.

In [ ]:
%pip uninstall -y transformers torch
%pip install transformers torch

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
%pip install transformers accelerate --upgrade

In [ ]:
%pip uninstall torch torchvision torchaudio -y
%pip cache purge
%pip install torch torchvision torchaudio

Pelatihan dan testing model BERT.

In [6]:
import torch
import transformers
import accelerate

print("PyTorch Version:", torch.__version__)
print("Transformers Version:", transformers.__version__)
print("Accelerate Version:", accelerate.__version__)
print("Is CUDA available:", torch.cuda.is_available())


c:\Users\Muhammad Fachri\miniconda3\envs\pytdml_py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Version: 2.3.1+cpu
Transformers Version: 4.50.1
Accelerate Version: 1.5.2
Is CUDA available: False


In [12]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertModel
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import numpy as np
from torch import nn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load data
df = pd.read_csv('processed_with_classified_sentiment.csv')

# Sample columns for data
df['review'] = df['processed_text'].apply(lambda tokens: ' '.join(tokens))

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Map sentiment labels to integer values
label_mapping = {"Negative": 0, "Neutral": 1, "Positive": 2}
df['label'] = df['sentiment_label'].map(label_mapping)  # Convert text labels to numerical values

#Custom Bert Model
class CustomBERTModel(nn.Module):
    def __init__(self, num_labels):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)  # Dropout to prevent overfitting
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)  # Classification layer
        self.loss_fn = nn.CrossEntropyLoss()  # Loss function

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Get the [CLS] token representation
        dropout_output = self.dropout(pooled_output)
        logits = self.classifier(dropout_output)  # Pass through classifier
        
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)  # Compute loss if labels are provided
        
        return {"loss": loss, "logits": logits}  # Return both loss and logits
    
# Define Dataset Class for Classification
class SentimentDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Prepare data
MAX_LEN = 128
BATCH_SIZE = 16

# Use classified labels
dataset = SentimentDataset(
    reviews=df['review'].tolist(),
    labels=df['label'].tolist(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

# Split dataset into training and testing
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Load BERT model for classification (3 sentiment classes)
model = CustomBERTModel(num_labels=3)

# Define custom compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)  # Get predicted labels
    
    # Ensure labels and logits are numpy arrays
    predictions = predictions if isinstance(predictions, np.ndarray) else predictions.cpu().numpy()
    labels = labels if isinstance(labels, np.ndarray) else labels.cpu().numpy()

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# Set up training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    output_dir='./results',
    num_train_epochs=5,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

# Define Trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics  # Add custom metrics
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)


c:\Users\Muhammad Fachri\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.483700,0.551902,0.833333,0.694444,0.833333,0.757576
2,0.354000,0.585172,0.833333,0.694444,0.833333,0.757576
3,0.418400,0.511108,0.833333,0.694444,0.833333,0.757576
4,0.473600,0.438551,0.851667,0.792982,0.851667,0.807948
5,0.481200,0.445293,0.850000,0.795508,0.850000,0.817520


c:\Users\Muhammad Fachri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Muhammad Fachri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Muhammad Fachri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

{'eval_loss': 0.4452926516532898, 'eval_accuracy': 0.85, 'eval_precision': 0.7955075075075075, 'eval_recall': 0.85, 'eval_f1': 0.8175204110080051, 'eval_runtime': 56.4143, 'eval_samples_per_second': 10.636, 'eval_steps_per_second': 0.674, 'epoch': 5.0}


c:\Users\Muhammad Fachri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Untuk ngesave model menjadi JSON.

In [13]:
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\vocab.txt',
 './saved_model\\added_tokens.json')

Evaluate model (jika diperlukan).

In [14]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer

# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("./saved_model")
tokenizer = BertTokenizer.from_pretrained("./saved_model")

# Reuse the existing trainer arguments
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Evaluate the model
metrics = trainer.evaluate()
print("Evaluation Results:", metrics)

#Hasil Evaluasi akhir, Hasil evaluasi di Cell sebelumnya juga sudah menunjukkan akurasi diatas 85%


Evaluation Results: {'eval_loss': 0.4452926516532898, 'eval_model_preparation_time': 0.0044, 'eval_accuracy': 0.85, 'eval_precision': 0.7955075075075075, 'eval_recall': 0.85, 'eval_f1': 0.8175204110080051, 'eval_runtime': 53.1216, 'eval_samples_per_second': 11.295, 'eval_steps_per_second': 0.715}


c:\Users\Muhammad Fachri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
